In [ ]:
# Restart the notebook and clear all state
import importlib
import sys

modules_to_remove = []
for module in sys.modules:
    if module.startswith('langgraph'):
        modules_to_remove.append(module)

for module in modules_to_remove:
    sys.modules.pop(module, None)

In [ ]:
# Allstate Auto Insurance Claim Settlement System
# Using Langchain/Langgraph Framework - Updated Implementation
#
# This script sets up the full claim processing system with support for
# uploaded documents and images, dynamic policy verification, and improved
# damage assessment with vision capabilities.

import os
import base64
import re
import json
import io
import tempfile
import PyPDF2
from typing import List, Dict, TypedDict, Annotated, Sequence, Tuple, Union
from IPython.display import Image, display, HTML

# Import Langchain and Langgraph components
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.schema.runnable import RunnableMap, RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import OpenAI, ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_core.messages import BaseMessage
from langgraph.graph import END, StateGraph
from langgraph.prebuilt import ToolNode

# === API Key Setup ===
def setup_api_key(api_key=None):
    """Set up the OpenAI API key."""
    if api_key:
        os.environ["OPENAI_API_KEY"] = api_key
    elif "OPENAI_API_KEY" not in os.environ:
        api_key = input("Enter your OpenAI API key: ")
        os.environ["OPENAI_API_KEY"] = api_key

# === State Definition ===
class ClaimState(TypedDict):
    policy_number: str
    image_path: str = None
    image_data: Union[bytes, str, None] = None
    policy_document_path: Union[bytes, str, None] = None
    messages: Annotated[Sequence[BaseMessage], "Chat history"]
    policy_verification: Dict
    damage_assessment: Dict
    claim_processing: Dict
    payment_processing: Dict
    notification: Dict

# === Helper Functions ===
def encode_image(image_path_or_data):
    """Encode an image as base64 string from file path or binary data."""
    if isinstance(image_path_or_data, bytes):
        return base64.b64encode(image_path_or_data).decode('utf-8')
    elif isinstance(image_path_or_data, str) and os.path.exists(image_path_or_data):
        with open(image_path_or_data, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    else:
        # Assume it's already a base64 string
        return image_path_or_data

def extract_policy_info_from_document(document_path):
    """
    Extract policy information from an uploaded PDF document.
    Falls back to hardcoded values if extraction fails.
    """
    policy_info = {
        'policy_number': '999 876 543',
        'effective_date': 'January 14, 2025',
        'vehicle': 'Honda Accord 2022',
        'vin': 'VYXGH98769879',
        'status': 'Active, not delinquent'
    }
    
    if not document_path:
        return policy_info
        
    try:
        # If the document was uploaded as bytes
        if isinstance(document_path, bytes):
            reader = PyPDF2.PdfReader(io.BytesIO(document_path))
        else:
            # If it's a file path
            reader = PyPDF2.PdfReader(document_path)
        
        # Extract text from all pages
        text = ""
        for page in reader.pages:
            text += page.extract_text() + "\n"
        
        # Extract policy number (assuming format like "Policy number: 999 876 543")
        policy_match = re.search(r"Policy\s+number:?\s*([\d\s]+)", text, re.IGNORECASE)
        if policy_match:
            policy_info['policy_number'] = policy_match.group(1).strip()
        
        # Extract effective date
        date_match = re.search(r"Policy\s+effective\s+date:?\s*([\w\s,]+\d{4})", text, re.IGNORECASE)
        if date_match:
            policy_info['effective_date'] = date_match.group(1).strip()
        
        # Extract vehicle info
        vehicle_match = re.search(r"Honda\s+Accord\s+\d{4}", text)
        if vehicle_match:
            policy_info['vehicle'] = vehicle_match.group(0).strip()
        
        # Extract VIN
        vin_match = re.search(r"VIN\s+([A-Z0-9]+)", text)
        if vin_match:
            policy_info['vin'] = vin_match.group(1).strip()
        
        # Look for delinquency indicators
        if "delinquent" in text.lower() or "past due" in text.lower() or "missed payment" in text.lower():
            policy_info['status'] = 'Delinquent'
        else:
            policy_info['status'] = 'Active, not delinquent'
            
    except Exception as e:
        print(f"Error extracting policy info: {e}")
        # Fall back to default values
    
    return policy_info

# === Initialize Models ===
def initialize_models():
    """Initialize the OpenAI models for each agent."""
    return {
        "policy_verifier": ChatOpenAI(model="gpt-4o-mini"),
        "damage_assessor": ChatOpenAI(model="gpt-4o-mini", max_tokens=1000),
        "claim_processor": ChatOpenAI(model="gpt-4o-mini", max_tokens=500),
        "payment_processor": ChatOpenAI(model="gpt-4o-mini", max_tokens=400),
        "notification": ChatOpenAI(model="gpt-4o-mini", max_tokens=300),
        "embeddings": OpenAIEmbeddings(model="text-embedding-3-small")
    }

# This function sets up the whole environment and returns all necessary components
def setup_claim_processing_system(api_key=None):
    """Set up the full claim processing system and return all necessary components."""
    # Setup API key
    setup_api_key(api_key)
    
    # Initialize models
    models = initialize_models()
    
    # Make models available globally
    global policy_verifier_model, damage_assessor_model, claim_processor_model
    global payment_processor_model, notification_model, embeddings
    
    policy_verifier_model = models["policy_verifier"]
    damage_assessor_model = models["damage_assessor"]
    claim_processor_model = models["claim_processor"]
    payment_processor_model = models["payment_processor"]
    notification_model = models["notification"]
    embeddings = models["embeddings"]
    
    print("✅ Allstate Claim Processing System initialized successfully!")
    
    return models

# Call setup by default
setup_claim_processing_system()

In [ ]:
from langgraph.graph import END, StateGraph
from langgraph.prebuilt import ToolNode

In [ ]:
# Allstate Auto Insurance Claim Settlement System
# Using Langchain/Langgraph Framework

# This notebook implements a multi-agent system for processing auto insurance claims
# Each agent handles a specific part of the claim settlement process
# The agents are connected in a graph using Langgraph to coordinate the workflow


In [ ]:
# Import necessary libraries
import os
import base64
from typing import List, Dict, TypedDict, Annotated, Sequence, Tuple, Union
from IPython.display import Image, display

In [ ]:
# Import Langchain and Langgraph components
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.schema.runnable import RunnableMap, RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import OpenAI, ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_core.messages import BaseMessage
from langgraph.graph import END, StateGraph
from langgraph.prebuilt import ToolNode

In [ ]:
os.environ["OPENAI_API_KEY"] = "key here"  # Replace with your actual OpenAI API key


In [ ]:
class ClaimState(TypedDict):
    policy_number: str
    image_path: str = None
    image_data: Union[bytes, str, None] = None
    policy_document_path: Union[bytes, str, None] = None
    messages: Annotated[Sequence[BaseMessage], "Chat history"]
    policy_verification: Dict
    damage_assessment: Dict
    claim_processing: Dict
    payment_processing: Dict
    notification: Dict

In [ ]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [ ]:
# Initialize OpenAI models
policy_verifier_model = ChatOpenAI(model="gpt-4o-mini")
damage_assessor_model = ChatOpenAI(model="gpt-4o-mini", max_tokens=1000)
claim_processor_model = ChatOpenAI(model="gpt-4o-mini", max_tokens=500)
payment_processor_model = ChatOpenAI(model="gpt-4o-mini", max_tokens=400)
notification_model = ChatOpenAI(model="gpt-4o-mini", max_tokens=300)

# Initialize embeddings model
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [ ]:

# Define the policy verification agent that processes the uploaded document
def policy_verification_agent(state: ClaimState) -> ClaimState:
    """
    Verifies if the policy number provided by the insured matches the records 
    and if the insured is not delinquent using the uploaded policy document.
    """
    # Extract policy information from PDF document if available
    policy_info = extract_policy_info_from_document(state.get("policy_document_path", None))
    
    # Policy verification prompt that incorporates the extracted policy data
    prompt = ChatPromptTemplate.from_messages([
        SystemMessage(content=f"""You are a policy verification agent for Allstate Auto Insurance.
        Your task is to verify if the policy number provided by the insured matches the records and if the insured is not delinquent.
        
        Based on the policy document, here are the valid policy details:
        - Policy Number: {policy_info.get('policy_number', '999 876 543')}
        - Effective Date: {policy_info.get('effective_date', 'January 14, 2025')}
        - Vehicle: {policy_info.get('vehicle', 'Honda Accord 2022')}
        - VIN: {policy_info.get('vin', 'VYXGH98769879')}
        - Policy Status: {policy_info.get('status', 'Active, not delinquent')}
        
        A policy is considered valid if it matches our records and is not expired.
        An insured is considered delinquent if they have missed payments or their policy has expired.
        
        Return a JSON object with the following fields:
        - policy_valid: true if the policy number matches our records, false otherwise
        - delinquent: false by default unless explicitly stated the policy is delinquent
        - verification_message: A message explaining the verification result
        - proceed_to_damage_assessment: true if both policy_valid is true AND delinquent is false, otherwise false
        """),
        MessagesPlaceholder(variable_name="messages"),
        HumanMessage(content=f"Please verify policy number: {state['policy_number']}")
    ])
    
    # Process the policy verification
    policy_verification_chain = prompt | policy_verifier_model
    result = policy_verification_chain.invoke({"messages": state["messages"], "policy_number": state["policy_number"]})
    
    # Extract the verification results from the model response
    try:
        # Look for JSON pattern in the response
        json_match = re.search(r'\{.*\}', result.content, re.DOTALL)
        if json_match:
            verification_json = json.loads(json_match.group())
        else:
            # If no JSON pattern found, try loading the whole content
            verification_json = json.loads(result.content)
    except json.JSONDecodeError:
        # If JSON parsing fails, create a default response based on policy number and extracted data
        valid_policy = state["policy_number"] == policy_info.get('policy_number', '999 876 543')
        verification_json = {
            "policy_valid": valid_policy,
            "delinquent": False,
            "verification_message": f"Policy {state['policy_number']} verification {'successful' if valid_policy else 'failed'}.",
            "proceed_to_damage_assessment": valid_policy
        }
    
    # Update the state with verification results
    state["policy_verification"] = verification_json
    
    # Add message to chat history
    state["messages"] = list(state["messages"]) + [
        AIMessage(content=f"Policy Verification Result: {verification_json['verification_message']}")
    ]
    
    return state


In [ ]:
def extract_policy_info_from_document(document_path):
    """
    Extract policy information from an uploaded PDF document.
    Falls back to hardcoded values if extraction fails.
    """
    policy_info = {
        'policy_number': '999 876 543',
        'effective_date': 'January 14, 2025',
        'vehicle': 'Honda Accord 2022',
        'vin': 'VYXGH98769879',
        'status': 'Active, not delinquent'
    }
    
    if not document_path:
        return policy_info
        
    try:
        # If the document was uploaded as bytes
        if isinstance(document_path, bytes):
            reader = PyPDF2.PdfReader(io.BytesIO(document_path))
        else:
            # If it's a file path
            reader = PyPDF2.PdfReader(document_path)
        
        # Extract text from all pages
        text = ""
        for page in reader.pages:
            text += page.extract_text() + "\n"
        
        # Extract policy number (assuming format like "Policy number: 999 876 543")
        policy_match = re.search(r"Policy\s+number:?\s*([\d\s]+)", text, re.IGNORECASE)
        if policy_match:
            policy_info['policy_number'] = policy_match.group(1).strip()
        
        # Extract effective date
        date_match = re.search(r"Policy\s+effective\s+date:?\s*([\w\s,]+\d{4})", text, re.IGNORECASE)
        if date_match:
            policy_info['effective_date'] = date_match.group(1).strip()
        
        # Extract vehicle info
        vehicle_match = re.search(r"Honda\s+Accord\s+\d{4}", text)
        if vehicle_match:
            policy_info['vehicle'] = vehicle_match.group(0).strip()
        
        # Extract VIN
        vin_match = re.search(r"VIN\s+([A-Z0-9]+)", text)
        if vin_match:
            policy_info['vin'] = vin_match.group(1).strip()
        
        # Look for delinquency indicators
        if "delinquent" in text.lower() or "past due" in text.lower() or "missed payment" in text.lower():
            policy_info['status'] = 'Delinquent'
        else:
            policy_info['status'] = 'Active, not delinquent'
            
    except Exception as e:
        print(f"Error extracting policy info: {e}")
        # Fall back to default values
    
    return policy_info

In [ ]:

def should_proceed_to_damage_assessment(state: ClaimState) -> str:
    """Determine if we should proceed to damage assessment based on policy verification."""
    if state["policy_verification"].get("proceed_to_damage_assessment", False):
        return "damage_assess"
    else:
        return "notify"

In [ ]:
# Define the main workflow graph
def create_claim_processing_graph():
    """Create the Langgraph workflow for claim processing."""
    
    # Initialize the graph with the state schema
    workflow = StateGraph(ClaimState)
    
    # Add nodes to the graph with different names than state keys
    workflow.add_node("policy_verify", policy_verification_agent)
    workflow.add_node("damage_assess", damage_assessment_agent)
    workflow.add_node("claim_process", claim_processing_agent)
    workflow.add_node("payment_process", payment_processing_agent)
    workflow.add_node("notify", notification_agent)
    
    # Define the edges in the workflow
    # Start with policy verification
    workflow.set_entry_point("policy_verify")
    
    # After policy verification, conditionally go to damage assessment or directly to notification
    workflow.add_conditional_edges(
        "policy_verify",
        should_proceed_to_damage_assessment,
        {
            "damage_assess": "damage_assess",
            "notify": "notify"
        }
    )
    
    # After damage assessment, proceed to claim processing
    workflow.add_edge("damage_assess", "claim_process")
    
    # After claim processing, proceed to payment processing
    workflow.add_edge("claim_process", "payment_process")
    
    # After payment processing, proceed to notification
    workflow.add_edge("payment_process", "notify")
    
    # After notification, end the workflow
    workflow.add_edge("notify", END)
    
    # Compile the graph
    return workflow.compile()

In [ ]:
# Function to process a claim
def process_claim(policy_number: str, image_path: str):
    """Process an insurance claim with the given policy number and damage image."""
    
    # Create the graph
    graph = create_claim_processing_graph()
    
    # Initialize the state
    initial_state = ClaimState(
        policy_number=policy_number,
        image_path=image_path,
        messages=[
            SystemMessage(content="Allstate Auto Insurance Claim Processing System"),
            HumanMessage(content=f"I'd like to submit a claim for policy number {policy_number}.")
        ],
        policy_verification={},
        damage_assessment={},
        claim_processing={},
        payment_processing={},
        notification={}
    )
    
    # Execute the graph with the initial state
    result = graph.invoke(initial_state)
    
    return result


In [ ]:
def display_claim_results(result):
    """Display the results of a claim processing run in a readable format."""
    print("\n" + "="*80)
    print(" "*30 + "CLAIM PROCESSING RESULTS")
    print("="*80)
    
    # Policy verification results
    print("\n📋 POLICY VERIFICATION:")
    print(f"Policy Number: {result.get('policy_number', 'N/A')}")
    print(f"Policy Valid: {result['policy_verification'].get('policy_valid', False)}")
    print(f"Delinquent: {result['policy_verification'].get('delinquent', False)}")
    print(f"Message: {result['policy_verification'].get('verification_message', 'N/A')}")
    print(f"Proceed to Damage Assessment: {result['policy_verification'].get('proceed_to_damage_assessment', False)}")
    
    # Damage assessment results
    if result['damage_assessment']:
        print("\n🔍 DAMAGE ASSESSMENT:")
        print(f"Assessment Completed: {result['damage_assessment'].get('assessment_completed', False)}")
        if result['damage_assessment'].get('assessment_completed', False):
            print(f"Damage Level: {result['damage_assessment'].get('damage_level', 'N/A')}")
            print(f"Recommended Payment: ${result['damage_assessment'].get('payment_amount', 0):,}")
            print(f"Details: {result['damage_assessment'].get('assessment_details', 'N/A')}")
    
    # Claim processing results
    if result['claim_processing']:
        print("\n✅ CLAIM PROCESSING:")
        print(f"Claim Processed: {result['claim_processing'].get('claim_processed', False)}")
        print(f"Claim Approved: {result['claim_processing'].get('claim_approved', False)}")
        
        # Only show approved amount if claim was approved
        if result['claim_processing'].get('claim_approved', False):
            print(f"Approved Amount: ${result['claim_processing'].get('approved_amount', 0):,}")
            
        print(f"Message: {result['claim_processing'].get('processing_message', 'N/A')}")
    
    # Payment processing results
    if result['payment_processing']:
        print("\n💰 PAYMENT PROCESSING:")
        print(f"Payment Authorized: {result['payment_processing'].get('payment_authorized', False)}")
        
        # Only show payment details if payment was authorized
        if result['payment_processing'].get('payment_authorized', False):
            print(f"Payment Amount: ${result['payment_processing'].get('payment_amount', 0):,}")
            print(f"Reference: {result['payment_processing'].get('payment_reference', 'N/A')}")
            
        print(f"Message: {result['payment_processing'].get('payment_message', 'N/A')}")
    
    # Notification results
    if result['notification']:
        print("\n📱 NOTIFICATION TO INSURED:")
        print(f"Type: {result['notification'].get('notification_type', 'N/A')}")
        print(f"Subject: {result['notification'].get('notification_subject', 'N/A')}")
        print(f"Message: {result['notification'].get('notification_message', 'N/A')}")
        print(f"Next Steps: {result['notification'].get('next_steps', 'N/A')}")
    
    print("\n" + "="*80)

In [ ]:
def process_claim_with_uploads(policy_number: str, damage_image, policy_document=None):
    """Process a claim with uploaded files
    
    Args:
        policy_number: The policy number to verify
        damage_image: Uploaded image file of car damage
        policy_document: Optional uploaded policy document PDF
    """
    # Read the image data
    if hasattr(damage_image, 'read'):
        # If it's a file-like object (from upload)
        image_data = damage_image.read()
    elif isinstance(damage_image, str) and os.path.exists(damage_image):
        # If it's a file path
        with open(damage_image, 'rb') as f:
            image_data = f.read()
    else:
        # Assume it's already binary data
        image_data = damage_image
    
    # Read the policy document if provided
    policy_doc_data = None
    if policy_document:
        if hasattr(policy_document, 'read'):
            # If it's a file-like object (from upload)
            policy_doc_data = policy_document.read()
        elif isinstance(policy_document, str) and os.path.exists(policy_document):
            # If it's a file path
            with open(policy_document, 'rb') as f:
                policy_doc_data = f.read()
        else:
            # Assume it's already binary data
            policy_doc_data = policy_document
    
    # Process the claim
    return process_claim(
        policy_number=policy_number,
        image_data=image_data,
        policy_document=policy_doc_data
    )

In [ ]:
def process_claim_with_uploads(policy_number: str, damage_image, policy_document=None):
    """Process a claim with uploaded files
    
    Args:
        policy_number: The policy number to verify
        damage_image: Uploaded image file of car damage
        policy_document: Optional uploaded policy document PDF
    """
    # Read the image data
    if hasattr(damage_image, 'read'):
        # If it's a file-like object (from upload)
        image_data = damage_image.read()
    elif isinstance(damage_image, str) and os.path.exists(damage_image):
        # If it's a file path
        with open(damage_image, 'rb') as f:
            image_data = f.read()
    else:
        # Assume it's already binary data
        image_data = damage_image
    
    # Read the policy document if provided
    policy_doc_data = None
    if policy_document:
        if hasattr(policy_document, 'read'):
            # If it's a file-like object (from upload)
            policy_doc_data = policy_document.read()
        elif isinstance(policy_document, str) and os.path.exists(policy_document):
            # If it's a file path
            with open(policy_document, 'rb') as f:
                policy_doc_data = f.read()
        else:
            # Assume it's already binary data
            policy_doc_data = policy_document
    
    # Process the claim
    return process_claim(
        policy_number=policy_number,
        image_data=image_data,
        policy_document=policy_doc_data
    )

In [ ]:
def run_updated_demo_cases():
    """Run demonstration cases for different scenarios with the updated implementation."""
    
    print("\n🚗 ALLSTATE AUTO INSURANCE CLAIM SETTLEMENT DEMO (UPDATED) 🚗\n")
    
    # Scenario 1: Valid policy with major damage - using the uploaded files
    print("\n\n🔍 SCENARIO 1: Valid policy with uploaded image and document")
    result1 = process_claim_with_uploads(
        policy_number="999 876 543",
        damage_image="./test_images/car_damage_major.jpg",
        policy_document="./policy_document.pdf"
    )
    display_claim_results(result1)
    
    # Scenario 2: Invalid policy number
    print("\n\n🔍 SCENARIO 2: Invalid policy number with uploaded document")
    result2 = process_claim_with_uploads(
        policy_number="123 456 789",  # Invalid policy number
        damage_image="./test_images/car_damage_minor.jpg",
        policy_document="./policy_document.pdf"
    )
    display_claim_results(result2)
    
    # Scenario 3: Major damage classification
    print("\n\n🔍 SCENARIO 3: Valid policy with major damage classification")
    # For this we'll use a specific major damage image
    result3 = process_claim_with_uploads(
        policy_number="999 876 543",
        damage_image="./test_images/car_damage_major.jpg"
    )
    display_claim_results(result3)
    
    # Scenario 4: Minor damage classification
    print("\n\n🔍 SCENARIO 4: Valid policy with minor damage classification")
    # For this we'll use a specific minor damage image
    result4 = process_claim_with_uploads(
        policy_number="999 876 543",
        damage_image="./test_images/car_damage_minor.jpg"
    )
    display_claim_results(result4)
    
    # Scenario 5: Delinquent policy (simulation)
    print("\n\n🔍 SCENARIO 5: Delinquent policy (simulation)")
    # We'll use a special function to simulate a delinquent policy
    result5 = process_delinquent_claim_with_uploads(
        policy_number="999 876 543",
        damage_image="./test_images/car_damage_moderate.jpg"
    )
    display_claim_results(result5)

def process_delinquent_claim_with_uploads(policy_number: str, damage_image, policy_document=None):
    """Process a claim with a delinquent policy for demonstration purposes."""
    # Read the image data
    if hasattr(damage_image, 'read'):
        # If it's a file-like object (from upload)
        image_data = damage_image.read()
    elif isinstance(damage_image, str) and os.path.exists(damage_image):
        # If it's a file path
        with open(damage_image, 'rb') as f:
            image_data = f.read()
    else:
        # Assume it's already binary data
        image_data = damage_image
    
    # Create a new graph instance
    graph = create_claim_processing_graph()
    
    # Initialize the state
    initial_state = ClaimState(
        policy_number=policy_number,
        image_data=image_data,
        policy_document_path=policy_document,
        messages=[
            SystemMessage(content="Allstate Auto Insurance Claim Processing System"),
            HumanMessage(content=f"I'd like to submit a claim for policy number {policy_number}.")
        ],
        policy_verification={},
        damage_assessment={},
        claim_processing={},
        payment_processing={},
        notification={}
    )
    
    # Execute just the policy verification step
    temp_workflow = StateGraph(ClaimState)
    temp_workflow.add_node("policy_verify", policy_verification_agent)
    temp_workflow.set_entry_point("policy_verify")
    temp_workflow.add_edge("policy_verify", END)
    temp_graph = temp_workflow.compile()
    
    # Get the initial policy verification
    initial_result = temp_graph.invoke(initial_state)
    
    # Override the policy verification to simulate a delinquent policy
    initial_result["policy_verification"] = {
        "policy_valid": True,
        "delinquent": True,
        "verification_message": "Policy is valid but delinquent due to missed payments.",
        "proceed_to_damage_assessment": False
    }
    
    # Add message to chat history
    initial_result["messages"] = list(initial_result["messages"]) + [
        AIMessage(content="Policy Verification Override: This policy is valid but marked as delinquent for demonstration purposes.")
    ]
    
    # Continue with the rest of the workflow
    modified_graph = create_claim_processing_graph()
    result = modified_graph.invoke(initial_result)
    
    return result

In [ ]:
# Define the payment processing agent
def payment_processing_agent(state: ClaimState) -> ClaimState:
    # Skip if claim processing didn't complete
    if not state["claim_processing"].get("claim_processed", False):
        state["payment_processing"] = {
            "payment_authorized": False,
            "payment_message": "Payment processing skipped due to incomplete claim processing."
        }
        return state
    
    # Payment processing prompt
    prompt = ChatPromptTemplate.from_messages([
        SystemMessage(content="""You are a payment processing agent for Allstate Auto Insurance.
        Your task is to authorize and process the payment for an approved claim.
        
        You should:
        1. Verify the claim has been approved
        2. Authorize the payment for the approved amount
        3. Generate a payment confirmation
        
        Return a JSON object with the following fields:
        - payment_authorized: true/false
        - payment_amount: The authorized payment amount in USD
        - payment_reference: A unique payment reference ID
        - payment_message: Confirmation message
        """),
        MessagesPlaceholder(variable_name="messages"),
        HumanMessage(content=f"""
        Please process payment for this claim:
        Claim processing results: {state['claim_processing']}
        """)
    ])
    
    # Process the payment
    payment_processing_chain = prompt | payment_processor_model
    result = payment_processing_chain.invoke({"messages": state["messages"]})
    
    # Extract the payment processing results
    import json
    import random
    import string
    
    # Generate a random payment reference ID
    payment_ref = ''.join(random.choices(string.ascii_uppercase + string.digits, k=8))
    
    # Try to parse JSON from the content
    try:
        payment_json = json.loads(result.content)
    except json.JSONDecodeError:
        # If JSON parsing fails, create a default response
        payment_json = {
            "payment_authorized": state["claim_processing"].get("claim_approved", False),
            "payment_amount": state["claim_processing"].get("approved_amount", 0),
            "payment_reference": payment_ref,
            "payment_message": "Payment has been processed successfully."
        }
    
    # Update the state with payment processing results
    state["payment_processing"] = payment_json
    
    # Add message to chat history
    state["messages"] = list(state["messages"]) + [
        AIMessage(content=f"Payment Processing: {payment_json['payment_message']} "
                          f"Amount: ${payment_json['payment_amount']}. "
                          f"Reference: {payment_json['payment_reference']}")
    ]
    
    return state


In [ ]:
def notification_agent(state: ClaimState) -> ClaimState:
    """
    Generates a notification to the insured about their claim status.
    Ensures that payment processing is completed before sending approval notification.
    """
    # Prepare notification content based on the entire process
    policy_verification = state["policy_verification"]
    payment_processing = state.get("payment_processing", {})
    claim_processing = state.get("claim_processing", {})
    
    # Determine notification type based on the processing chain
    if not policy_verification.get("policy_valid", False):
        notification_type = "claim_denied"
        reason = "invalid policy"
    elif policy_verification.get("delinquent", False):
        notification_type = "claim_denied"
        reason = "policy delinquent"
    elif not claim_processing.get("claim_approved", False):
        notification_type = "claim_denied"
        reason = "claim not approved"
    elif not payment_processing.get("payment_authorized", False):
        notification_type = "claim_denied"
        reason = "payment not authorized"
    else:
        notification_type = "claim_approved"
        reason = "all verifications passed"
    
    # Only proceed with approval notification if payment has been authorized
    payment_completed = payment_processing.get("payment_authorized", False)
    
    # Notification prompt
    prompt = ChatPromptTemplate.from_messages([
        SystemMessage(content=f"""You are a notification agent for Allstate Auto Insurance.
        Your task is to generate a notification to the insured about their claim status.
        
        The notification should be clear, professional, and provide all necessary information.
        
        Important: For an approval notification, payment processing MUST be completed first.
        Current payment status: {"Completed" if payment_completed else "Not completed"}
        
        Return a JSON object with the following fields:
        - notification_type: "{notification_type}"
        - notification_subject: A subject line for the notification
        - notification_message: The full notification message
        - next_steps: Instructions for the insured on what to do next
        """),
        MessagesPlaceholder(variable_name="messages"),
        HumanMessage(content=f"""
        Please generate a notification for the insured with the following information:
        
        Notification type: {notification_type}
        Reason: {reason}
        
        Policy verification: {state['policy_verification']}
        Damage assessment: {state.get('damage_assessment', {})}
        Claim processing: {state.get('claim_processing', {})}
        Payment processing: {state.get('payment_processing', {})}
        """)
    ])
    
    # Generate the notification
    notification_chain = prompt | notification_model
    result = notification_chain.invoke({"messages": state["messages"]})
    
    # Extract the notification results
    try:
        # Look for JSON pattern in the response
        json_match = re.search(r'\{.*\}', result.content, re.DOTALL)
        if json_match:
            notification_json = json.loads(json_match.group())
        else:
            # If no JSON pattern found, try loading the whole content
            notification_json = json.loads(result.content)
    except json.JSONDecodeError:
        # If JSON parsing fails, create a default response
        notification_json = {
            "notification_type": notification_type,
            "notification_subject": f"Your Allstate Claim - {notification_type.replace('_', ' ').title()}",
            "notification_message": f"Your claim has been {notification_type.replace('_', ' ')}.",
            "next_steps": "Please contact customer service if you have any questions."
        }
    
    # Ensure the notification aligns with payment status
    if notification_type == "claim_approved" and not payment_completed:
        # Override to pending if payment isn't completed
        notification_json["notification_type"] = "claim_pending"
        notification_json["notification_subject"] = "Your Allstate Claim - Processing"
        notification_json["notification_message"] = "Your claim is being processed. Payment authorization is pending."
        notification_json["next_steps"] = "Please wait for final approval notification once payment processing is complete."
    
    # Update the state with notification results
    state["notification"] = notification_json
    
    # Add message to chat history
    state["messages"] = list(state["messages"]) + [
        AIMessage(content=f"Notification to Insured: {notification_json['notification_subject']}\n\n"
                          f"{notification_json['notification_message']}\n\n"
                          f"Next Steps: {notification_json['next_steps']}")
    ]
    
    return state

In [ ]:

# Define the damage assessment agent
def damage_assessment_agent(state: ClaimState) -> ClaimState:
    # Skip if policy verification didn't pass
    if not state["policy_verification"].get("proceed_to_damage_assessment", False):
        state["damage_assessment"] = {
            "assessment_completed": False,
            "assessment_message": "Damage assessment skipped due to policy verification failure."
        }
        return state
    
    # Encode the image for the vision model
    base64_image = encode_image(state["image_path"])
    
    # Damage assessment prompt
    prompt = ChatPromptTemplate.from_messages([
        SystemMessage(content="""You are a damage assessment agent for Allstate Auto Insurance.
        Your task is to analyze the car damage image and classify the level of damage (minor, moderate, or major).
        
        Minor damage: Small scratches, dents, or cosmetic issues that don't affect vehicle functionality.
        Moderate damage: Visible body damage, broken lights, or damages that may affect some vehicle functionality.
        Major damage: Significant structural damage, deployed airbags, or damages that render the vehicle unsafe to drive.
        
        Based on your assessment, you should also determine a recommended payment amount:
        - Minor damage: $500-$1,500
        - Moderate damage: $1,500-$5,000
        - Major damage: $5,000-$20,000
        
        Return a JSON object with the following fields:
        - damage_level: "minor", "moderate", or "major"
        - payment_amount: Recommended payment amount in USD
        - assessment_details: Brief description of the damage
        - assessment_completed: true
        """),
        MessagesPlaceholder(variable_name="messages"),
        HumanMessage(content=[
            {"type": "text", "text": "Please assess the damage in this image:"},
            {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
        ])
    ])
    
    # Process the damage assessment
    damage_assessment_chain = prompt | damage_assessor_model
    result = damage_assessment_chain.invoke({"messages": state["messages"]})
    
    # Extract the assessment results
    import json
    # Try to parse JSON from the content
    try:
        assessment_json = json.loads(result.content)
    except json.JSONDecodeError:
        # If JSON parsing fails, create a default response
        assessment_json = {
            "damage_level": "moderate",
            "payment_amount": 3000,
            "assessment_details": "Vehicle appears to have moderate damage to the body panels.",
            "assessment_completed": True
        }
    
    # Update the state with assessment results
    state["damage_assessment"] = assessment_json
    
    # Add message to chat history
    state["messages"] = list(state["messages"]) + [
        AIMessage(content=f"Damage Assessment: {assessment_json['damage_level']} damage. "
                          f"Recommended payment: ${assessment_json['payment_amount']}. "
                          f"Details: {assessment_json['assessment_details']}")
    ]
    
    return state

In [ ]:


# Define the claim processing agent
def claim_processing_agent(state: ClaimState) -> ClaimState:
    # Skip if damage assessment didn't complete
    if not state["damage_assessment"].get("assessment_completed", False):
        state["claim_processing"] = {
            "claim_processed": False,
            "processing_message": "Claim processing skipped due to incomplete damage assessment."
        }
        return state
    
    # Claim processing prompt
    prompt = ChatPromptTemplate.from_messages([
        SystemMessage(content="""You are a claim processing agent for Allstate Auto Insurance.
        Your task is to review the policy verification and damage assessment results,
        and make a final determination on the claim.
        
        You should:
        1. Review the policy verification results
        2. Review the damage assessment
        3. Make a final determination on the claim's validity
        4. Authorize the payment amount recommended by the damage assessment
        
        Return a JSON object with the following fields:
        - claim_approved: true/false
        - approved_amount: The payment amount in USD
        - processing_message: Explanation of the decision
        - claim_processed: true
        """),
        MessagesPlaceholder(variable_name="messages"),
        HumanMessage(content=f"""
        Please process this claim based on:
        Policy verification: {state['policy_verification']}
        Damage assessment: {state['damage_assessment']}
        """)
    ])
    
    # Process the claim
    claim_processing_chain = prompt | claim_processor_model
    result = claim_processing_chain.invoke({"messages": state["messages"]})
    
    # Extract the claim processing results
    import json
    # Try to parse JSON from the content
    try:
        processing_json = json.loads(result.content)
    except json.JSONDecodeError:
        # If JSON parsing fails, create a default response
        processing_json = {
            "claim_approved": True,
            "approved_amount": state["damage_assessment"].get("payment_amount", 0),
            "processing_message": "Claim has been reviewed and processed successfully.",
            "claim_processed": True
        }
    
    # Update the state with claim processing results
    state["claim_processing"] = processing_json
    
    # Add message to chat history
    state["messages"] = list(state["messages"]) + [
        AIMessage(content=f"Claim Processing: {processing_json['processing_message']} "
                          f"Approved amount: ${processing_json['approved_amount']}"

In [ ]:
# If running this notebook directly, run the demos
if __name__ == "__main__":
    run_demo_cases()
